RUN000 - Setup Master Pool runner infrastructure
================================================

Description
-----------

Create the Big Data Cluster SQL Master Pool objects to enable saving
results.

NOTE: This notebook uses the Python kernel, not the SQL kernel, so that
it can be run in a ‘single click’ on all machines. (The SQL kernel
requires a PowerShell module to be installed, that can’t be guaranteed
to be present on all machines).

### Parameters

Passwords are generated for the Logins to enable metrics to be saved
(insert\_metrics) by the run002 notebook, and to be read by the Grafana
dashboards. These Passwords are only used if the Logins do not already
exist.

In [ ]:
import os
import secrets
import string

# Generate passwords
#
alphabet = string.ascii_letters + string.digits + string.punctuation

os.environ["runner_insert_metric_password"] = ''.join(secrets.choice(alphabet) for i in range(20)) # for a 20-character password
os.environ["runner_db_reader_password"] = ''.join(secrets.choice(alphabet) for i in range(20)) # for a 20-character password

os.environ["runner_insert_metric_password"] = os.environ["runner_insert_metric_password"].replace('"', "-").replace("'", "_")
os.environ["runner_db_reader_password"] = os.environ["runner_db_reader_password"].replace('"', "-").replace("'", "_")

print("Passwords generated")

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

first_run = True
rules = None
debug_logging = False

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # apply expert rules (to run follow-on notebooks), based on output
                #
                if rules is not None:
                    apply_expert_rules(line_decoded)

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    return output
                                else:
                                    return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    """Load a json file from disk and return the contents"""

    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():
    """Load any 'expert rules' from the metadata of this notebook (.ipynb) that should be applied to the stderr of the running executable"""

    # Load this notebook as json to get access to the expert rules in the notebook metadata.
    #
    try:
        j = load_json("run000-setup-infrastructure.ipynb")
    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?
    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "expanded_rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["expanded_rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):
    """Determine if the stderr line passed in, matches the regular expressions for any of the 'expert rules', if so
    inject a 'HINT' to the follow-on SOP/TSG to run"""

    global rules

    for rule in rules:
        notebook = rule[1]
        cell_type = rule[2]
        output_type = rule[3] # i.e. stream or error
        output_type_name = rule[4] # i.e. ename or name 
        output_type_value = rule[5] # i.e. SystemExit or stdout
        details_name = rule[6]  # i.e. evalue or text 
        expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

        if debug_logging:
            print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

        if re.match(expression, line, re.DOTALL):

            if debug_logging:
                print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

            match_found = True

            display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))




print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'], 'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use']}
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb']], 'azdata': [['azdata login', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Error processing command: "ApiError', 'TSG110 - Azdata returns ApiError', '../repair/tsg110-azdata-returns-apierror.ipynb'], ['Error processing command: "ControllerError', 'TSG036 - Controller logs', '../log-analyzers/tsg036-get-controller-logs.ipynb'], ['ERROR: 500', 'TSG046 - Knox gateway logs', '../log-analyzers/tsg046-get-knox-logs.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ["Can't open lib 'ODBC Driver 17 for SQL Server", 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb']]}
install_hint = {'kubectl': ['SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb'], 'azdata': ['SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb']}

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Get the controller username and password

Get the controller username and password from the Kubernetes Secret
Store and place in the required AZDATA\_USERNAME and AZDATA\_PASSWORD
environment variables.

In [ ]:
# Place controller secret in AZDATA_USERNAME/AZDATA_PASSWORD environment variables

import os, base64

os.environ["AZDATA_USERNAME"] = run(f'kubectl get secret/controller-login-secret -n {namespace} -o jsonpath={{.data.username}}', return_output=True)
os.environ["AZDATA_USERNAME"] = base64.b64decode(os.environ["AZDATA_USERNAME"]).decode('utf-8')

os.environ["AZDATA_PASSWORD"] = run(f'kubectl get secret/controller-login-secret -n {namespace} -o jsonpath={{.data.password}}', return_output=True)
os.environ["AZDATA_PASSWORD"] = base64.b64decode(os.environ["AZDATA_PASSWORD"]).decode('utf-8')


print(f"Controller username '{os.environ['AZDATA_USERNAME']}' and password stored in environment variables")

### Create Kubernetes secrets and SQL logins

Create a T/SQL Login to insert metrics (done in the run002 notebook),
and read metrics from Grafana.

In [ ]:
import json

def create_tsql_login_if_not_exists(name, password_env_var):
    output = run(f"""azdata sql query -q "SELECT COUNT(*) as login_exists FROM sys.syslogins where name = '##{name}##'" """, return_output=True)

    login_exists = bool(json.loads(output)[0]["login_exists"])

    if not login_exists:

        # Delete K8s secret if previously created
        # 
        secret = run(f"kubectl get secrets --field-selector metadata.name={name.replace('_', '-')}-secret -n {namespace} --no-headers -o jsonpath={{.items}}", return_output=True)

        if secret != "[]":
            run(f"kubectl delete secret {name.replace('_', '-')}-secret -n {namespace}")

        run(f"""kubectl create secret generic {name.replace('_', '-')}-secret -n {namespace} --from-literal=username=##{name}## --from-literal=password={os.environ[password_env_var]}""")

        sql = f"""CREATE LOGIN ##{name}## WITH PASSWORD = '{os.environ[password_env_var]}'"""

        run(f'azdata sql query --database master -q "{sql}"')

        # Clear out the password
        #
        os.environ[password_env_var] = ""

create_tsql_login_if_not_exists('runner_insert_metric_login', "runner_insert_metric_password")
create_tsql_login_if_not_exists('runner_db_reader_login', "runner_db_reader_password")

### Create the `runner` database

The metrics will be held in a database named `runner`

In [ ]:
sql = f"""
IF DB_ID('runner') IS NULL
BEGIN
  CREATE DATABASE [runner]
END
"""

run(f'azdata sql query --database master -q "{sql}"')

### Create the SQL tables

Create the required T/SQL tables to hold the metrics. The schema is
normalized to optimize performance (dense records), and views are
provided to consume the data in dashboards.

-   `tbl_notebook` – generates an internal `id` for each notebook file
    name received
-   `tbl_app` – generates an internal `id` for each ‘app-deploy’ app
    name/version received. It also notes if the app-deploy is used as a
    canary (to enable dashboards to optimize displays)
-   `tbl_session` – generate an `id` for each session. A session is a
    single invocation of all the notebooks in an app-deploy.
-   `tbl_metrics` – the fully normalized table containing all the
    metrics of notebook execution

In [ ]:
sql = f"""
IF NOT EXISTS (SELECT *
              FROM INFORMATION_SCHEMA.TABLES
              WHERE TABLE_SCHEMA = 'dbo'
              AND TABLE_NAME = 'tbl_notebook')
BEGIN
    CREATE TABLE [dbo].[tbl_notebook](
        [id] [int] NOT NULL IDENTITY,
        [name] [sysname] NOT NULL,
        [created] [datetime2](7) NOT NULL DEFAULT GETDATE()
        CONSTRAINT [PK__notebook__id] PRIMARY KEY CLUSTERED 
        (
            [id] ASC
        )
    )
END

IF NOT EXISTS (SELECT *
              FROM INFORMATION_SCHEMA.TABLES
              WHERE TABLE_SCHEMA = 'dbo'
              AND TABLE_NAME = 'tbl_app')
BEGIN
    CREATE TABLE [dbo].[tbl_app](
        [id] [int] NOT NULL IDENTITY,
        [name] [sysname] NOT NULL,
        [version] [sysname] NOT NULL,
        [is_canary] [bit] NOT NULL,
        [created] [datetime2](7) NOT NULL DEFAULT GETDATE()
        CONSTRAINT [PK__app__id] PRIMARY KEY CLUSTERED 
        (
            [id] ASC
        )
    )
END

IF NOT EXISTS (SELECT *
              FROM INFORMATION_SCHEMA.TABLES
              WHERE TABLE_SCHEMA = 'dbo'
              AND TABLE_NAME = 'tbl_session')
BEGIN
    CREATE TABLE [dbo].[tbl_session](
        [id] [int] NOT NULL IDENTITY,
        [start] [datetime2](7) NOT NULL,
        [app_id] [int] NOT NULL CONSTRAINT FK__session__app_id FOREIGN KEY REFERENCES [dbo].[tbl_app](id) ON DELETE CASCADE
        CONSTRAINT [PK__session__id] PRIMARY KEY CLUSTERED 
        (
            [id] ASC
        )
    )
END

IF NOT EXISTS (SELECT *
              FROM INFORMATION_SCHEMA.TABLES
              WHERE TABLE_SCHEMA = 'dbo'
              AND TABLE_NAME = 'tbl_metrics')
BEGIN
    CREATE TABLE [dbo].[tbl_metrics](
        [session_id] [int] NOT NULL CONSTRAINT FK__metrics__session_id FOREIGN KEY REFERENCES [dbo].[tbl_session](id) ON DELETE CASCADE,
        [step] [tinyint] NOT NULL,
        [notebook_id] [int] NOT NULL CONSTRAINT FK__metrics__notebook_id FOREIGN KEY REFERENCES [dbo].[tbl_notebook](id) ON DELETE CASCADE,
        [error_level] [tinyint] NOT NULL,
        [base_notebook_id] [int] NULL CONSTRAINT FK__metrics__base_notebook_id FOREIGN KEY REFERENCES [dbo].[tbl_notebook](id),
        [parent_notebook_id] [int] NULL CONSTRAINT FK__metrics__parent_notebook_id FOREIGN KEY REFERENCES [dbo].[tbl_notebook](id),
        [html_notebook_included] [bit] NOT NULL,
        [start] [datetime2](7) NOT NULL,
        [end] [datetime2](7) NOT NULL
        CONSTRAINT [PK__metrics__session_id__start] PRIMARY KEY CLUSTERED 
        (
            [session_id] ASC,
            [start] ASC
        )
    )
END
"""

run(f'azdata sql query --database runner -q "{sql}"')

### Create `insert_metric` procedure

The `insert metric` procedure is called from the run002 notebook.

In [ ]:
sql = f"""
CREATE OR ALTER PROCEDURE [dbo].[insert_metric]
    @session_start DATETIME2, 
    @app_name SYSNAME,
    @app_version SYSNAME,
    @step TINYINT,
    @notebook_name NVARCHAR(max),
    @error_level TINYINT,
    @base_notebook_name NVARCHAR(max),
    @parent_notebook_name NVARCHAR(max),
    @html_notebook_included BIT,
    @start DATETIME2,
    @end DATETIME2
AS
BEGIN
    SET NOCOUNT ON
    SET XACT_ABORT ON
    BEGIN TRAN

        /* Get the app_id from the app_name/app_version */
        DECLARE @app_id INT = (SELECT id FROM [dbo].[tbl_app] WHERE [name] = @app_name AND [version] = @app_version)
        IF @app_id IS NULL
        BEGIN
            INSERT INTO [dbo].[tbl_app] VALUES (@app_name, @app_version, 0, DEFAULT)
            SET @app_id = SCOPE_IDENTITY()
        END

        /* Get the session_id from the session_start/app_name/app_version */
        DECLARE @session_id INT = (SELECT id FROM [dbo].[tbl_session] WHERE [start] = @session_start and [app_id] = @app_id)
        IF @session_id IS NULL
        BEGIN
            INSERT INTO [dbo].[tbl_session] VALUES (@session_start, @app_id)
            SET @session_id = SCOPE_IDENTITY()
        END

        /* Get the notebook_id for the notebook_name */
        DECLARE @notebook_id INT = (SELECT id FROM [dbo].[tbl_notebook] WHERE [name] = @notebook_name)
        IF @notebook_id IS NULL
        BEGIN
            INSERT INTO [dbo].[tbl_notebook] VALUES (@notebook_name, DEFAULT)
            SET @notebook_id = SCOPE_IDENTITY()
        END

        /* Get the notebook_id for the base_notebook_name */
        DECLARE @base_notebook_id INT = NULL
        IF @base_notebook_name <> ''
        BEGIN
            SET @base_notebook_id = (SELECT id FROM [dbo].[tbl_notebook] WHERE [name] = @base_notebook_name)
            IF @base_notebook_id IS NULL
            BEGIN
                INSERT INTO [dbo].[tbl_notebook] VALUES (@base_notebook_name, DEFAULT)
                SET @base_notebook_id = SCOPE_IDENTITY()
            END
        END

        /* Get the notebook_id for the parent_notebook_name */
        DECLARE @parent_notebook_id INT = NULL
        IF @parent_notebook_name <> ''
        BEGIN
            SET @parent_notebook_id = (SELECT id FROM [dbo].[tbl_notebook] WHERE [name] = @parent_notebook_name)
            IF @parent_notebook_id IS NULL
            BEGIN
                INSERT INTO [dbo].[tbl_notebook] VALUES (@parent_notebook_name, DEFAULT)
                SET @parent_notebook_id = SCOPE_IDENTITY()
            END
        END

        INSERT [dbo].[tbl_metrics] VALUES(@session_id, @step, @notebook_id, @error_level, @base_notebook_id, @parent_notebook_id, @html_notebook_included, @start, @end)

    COMMIT
END
"""

run(f'azdata sql query --database runner -q "{sql}"')

### Create views

These views are used to power the Grafana dashboards

In [ ]:
sql = f"""
CREATE OR ALTER VIEW [dbo].[sessions]
AS
    SELECT s.id, s.start as session_start, a.name as app_name, a.version as app_version, a.is_canary
    FROM [dbo].[tbl_session] s
    JOIN [dbo].[tbl_app] a ON s.app_id = a.id
"""

run(f'azdata sql query --database runner -q "{sql}"')

In [ ]:
sql = f"""
CREATE OR ALTER VIEW [dbo].[metrics]
AS
    SELECT 
        s.[session_start],
        s.[app_name], 
        s.[app_version],
        m.[step],
        n.[name],
        m.[error_level],
        p.[name] as [parent_name],
        m.[start],
        m.[end],
        'app-deploy/' + /* Build up the URL to this notebook in HDFS */
            s.[app_name] + '-' + s.[app_version] +
            '/logs/' +
            FORMAT(DATEPART(yy, s.[session_start]),'D4') + '/' +
            FORMAT(DATEPART(mm, s.[session_start]),'D2') + '/' +
            FORMAT(DATEPART(dd, s.[session_start]),'D2') + '/' +
            FORMAT(DATEPART(hh, s.[session_start]),'D2') + '-' +
            FORMAT(DATEPART(mi, s.[session_start]),'D2') + '-' +
            CASE m.[error_level] WHEN 0 THEN 'SUCCESS-' ELSE 'FAILURE-' END + 
            'step' + FORMAT(m.[step],'D3') + '-' +
            n.[name] +
            CASE m.html_notebook_included WHEN 1 THEN '.html' ELSE '.ipynb' END as [notebook]
    FROM dbo.tbl_metrics m
    JOIN dbo.sessions s on m.session_id = s.id
    JOIN dbo.tbl_notebook n on m.notebook_id = n.id
    LEFT OUTER JOIN dbo.tbl_notebook p on m.parent_notebook_id = p.id
"""

run(f'azdata sql query --database runner -q "{sql}"')

### Create canary\_stats function

Grafana dashboard don’t work with CTEs (Common Table Expressions),
therefore creating Functions here.

In [ ]:
sql = f"""
CREATE OR ALTER FUNCTION canary_stats (@hours INT)
RETURNS TABLE
AS
RETURN
    WITH last_hours([app_name], [passed], [failed])
    AS
    (
        SELECT  [app_name],
            (SELECT count(*) FROM [metrics] r2 WHERE r1.app_name = r2.app_name AND r2.error_level = 0 AND app_name LIKE 'canary-%' AND start > DATEADD(hour, @hours, GETDATE())) AS [passed],
            (SELECT count(*) FROM [metrics] r2 WHERE r1.app_name = r2.app_name AND r2.error_level <> 0 AND app_name LIKE 'canary-%' AND start > DATEADD(hour, @hours, GETDATE())) AS [failed]
        FROM [metrics] r1
        WHERE app_name LIKE 'canary-%' AND start > DATEADD(hour, @hours, GETDATE())
        GROUP BY [app_name]
    )
    SELECT [app_name] as Canary, [Passed], [Failed], [passed] + [failed] as Total, CONVERT(DECIMAL(5, 2), ([passed] * 1.0) / ([passed] + [failed]) * 100.0) as [Percent]
    FROM last_hours
"""

run(f'azdata sql query --database runner -q "{sql}"')

### Create maintenance procs

Use these procs to clear up the runner database:

-   `delete_app_metrics` – if there is an app you no longer want to
    appear in dashboards.

In [ ]:
sql = f"""
CREATE OR ALTER PROCEDURE [dbo].[delete_app_metrics]
    @app_name SYSNAME,
    @app_version SYSNAME
AS
BEGIN
    SET NOCOUNT ON
    SET XACT_ABORT ON
    BEGIN TRAN
        DELETE [dbo].[tbl_app] WHERE name = @app_name and version = @app_version
    COMMIT
END
"""

run(f'azdata sql query --database runner -q "{sql}"')

sql = f"""
CREATE OR ALTER PROCEDURE [dbo].[delete_notebook_metrics] @name SYSNAME AS
BEGIN
    SET NOCOUNT ON
    SET XACT_ABORT ON 
    
    BEGIN TRAN
        DECLARE @notebook_id INT = (SELECT id FROM tbl_notebook WHERE name = @name)

        /* Update self referencing columns to NULL, to allow DELETE CASCASE */
        UPDATE tbl_metrics
        SET parent_notebook_id = NULL 
        WHERE parent_notebook_id = @notebook_id

        UPDATE tbl_metrics
        SET base_notebook_id = NULL 
        WHERE base_notebook_id = @notebook_id

        DELETE [dbo].[tbl_notebook] WHERE name = @name     
    COMMIT
END
"""

run(f'azdata sql query --database runner -q "{sql}"')

### Set up permissions

In [ ]:
sql = f"""
IF NOT EXISTS (SELECT * FROM sys.sysusers WHERE name = '##runner_db_reader_login##')
BEGIN
    CREATE USER ##runner_db_reader_login## FOR LOGIN ##runner_db_reader_login##
    ALTER ROLE db_datareader ADD MEMBER ##runner_db_reader_login##
END

IF NOT EXISTS (SELECT * FROM sys.sysusers WHERE name = '##runner_insert_metric_login##')
BEGIN
    CREATE USER ##runner_insert_metric_login## FOR LOGIN ##runner_insert_metric_login##
    GRANT CONNECT TO [##runner_insert_metric_login##]
    GRANT EXECUTE ON [dbo].[insert_metric] TO [##runner_insert_metric_login##]
END
"""

run(f'azdata sql query --database runner -q "{sql}"')

In [ ]:
print('Notebook execution complete.')

Related
-------

-   [RUN003 - Run expert
    rules](../notebook-runner/run003-run-expert-rules.ipynb)

-   [OPR001 - Create
    app-deploy](../notebook-o16n/opr001-create-app-deploy.ipynb)

-   [OPR600 - Monitor infrastructure
    (Kubernetes)](../notebook-o16n/opr600-monitor-infrastructure-k8s.ipynb)

-   [OPR900 - Troubleshoot run
    app-deploy](../notebook-o16n/opr900-troubleshoot-run-app-deploy.ipynb)

-   [RUN999 - Cleanup Master Pool runner
    infrastructure](../notebook-runner/run999-cleanup-infrastructure.ipynb)